# LAB 1

In [2]:
import datetime
import requests
import pandas as pd
import hopsworks
import datetime
from pathlib import Path
from functions import util
import json
import re
import os
import warnings
warnings.filterwarnings("ignore")

## 1.Backfill feature pipeline

Write a backfill feature pipeline that downloads historical weather data (ideally >1
year of data), loads a csv file with historical air quality data (downloaded from
https://aqicn.org) and registers them as 2 Feature Groups with Hopsworks.

### 1.1 Import weather data on one year at Stockholm

In [28]:
import requests
import pandas as pd
import json

# Coordinate of Stockholm and date of historical datas
latitude = 59.3293
longitude = 18.0686
start_date = "2022-01-01"  # Start date of the data
end_date = "2022-12-31"    # End date of the data

# URL Open Meteo
weather_url = f"https://archive-api.open-meteo.com/v1/era5?latitude={latitude}&longitude={longitude}&start_date={start_date}&end_date={end_date}&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m,wind_direction_10m,wind_gusts_10m"

#Request the datas
response = requests.get(weather_url)
weather_data = response.json()

# convert to pd
df_weather = pd.DataFrame({
    "time": pd.to_datetime(weather_data["hourly"]["time"]),
    "temperature_2m": weather_data["hourly"]["temperature_2m"],
    "relative_humidity_2m": weather_data["hourly"]["relative_humidity_2m"],
    "wind_speed_10m": weather_data["hourly"]["wind_speed_10m"],
    "wind_direction_10m": weather_data["hourly"]["wind_direction_10m"],
    "wind_gusts_10m": weather_data["hourly"]["wind_gusts_10m"]
})

print(df_weather.head())



                 time  temperature_2m  relative_humidity_2m  wind_speed_10m  \
0 2022-01-01 00:00:00            -0.4                    98            10.0   
1 2022-01-01 01:00:00            -0.3                    98            11.5   
2 2022-01-01 02:00:00             0.1                    99            16.2   
3 2022-01-01 03:00:00             0.5                    97            17.5   
4 2022-01-01 04:00:00             0.0                    96            17.1   

   wind_direction_10m  wind_gusts_10m  
0                 322            28.1  
1                 314            22.0  
2                 328            32.4  
3                 352            35.3  
4                 353            36.4  


### 1.2 Import air quality data on 5 years at Stockholm

In [29]:
air_quality_df = pd.read_csv("../../data/stockholm-hornsgatan 108 gata-air-quality.csv")

air_quality_df['date'] = pd.to_datetime(air_quality_df['date'], format='%Y/%m/%d')

air_quality_df = air_quality_df.rename(columns={'date': 'time'})

air_quality_df = air_quality_df.drop(columns=['no2'])


air_quality_df.head()

,time,pm25,pm10
0,2024-11-01,8,3
1,2024-11-02,5,3
2,2024-11-03,6,6
3,2024-11-04,7,4
4,2024-11-05,11,6


### 1.3 Connection to hopsworks

In [3]:
import hopsworks

with open('../../data/hopsworks-api-key.txt', 'r') as file:
    os.environ["HOPSWORKS_API_KEY"] = file.read().rstrip()

In [31]:


project = hopsworks.login(project="ID2223LAB1KTH")
fs = project.get_feature_store()


Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1170583
Connected. Call `.close()` to terminate connection gracefully.


In [32]:
# Create a weather data feature group
weather_fg = fs.create_feature_group(
    name="stockholm_weather",
    version=1,
    description="Weather data for Stockholm including temperature, humidity, wind speed, and wind direction",
    primary_key=["time"],  
    event_time="time" 
)


weather_fg.insert(df_weather)

print("Weather feature group created and data inserted successfully.")


RestAPIError: Metadata operation error: (url: https://c.app.hopsworks.ai/hopsworks-api/api/project/1170583/featurestores/1161286/featuregroups). Server response: 
HTTP code: 400, HTTP reason: Bad Request, body: b'{"errorCode":270089,"usrMsg":"project: ID2223LAB1KTH, featurestoreId: 1161286","errorMsg":"The feature group you are trying to create does already exist."}', error code: 270089, error msg: The feature group you are trying to create does already exist., user msg: project: ID2223LAB1KTH, featurestoreId: 1161286

In [33]:

air_quality_fg = fs.create_feature_group(
    name="stockholm_air_quality",
    version=1,
    description="Air quality data for Stockholm with PM2.5 concentrations",
    primary_key=["time"],  # 'time' column as the primary key
    event_time="time"      # Specify 'time' as the event time
)


air_quality_fg.insert(air_quality_df)

print("Air quality feature group created and data inserted successfully.")


Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1170583/fs/1161286/fg/1347917


Uploading Dataframe: 0.00% |          | Rows 0/2556 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: stockholm_air_quality_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1170583/jobs/named/stockholm_air_quality_1_offline_fg_materialization/executions
Air quality feature group created and data inserted successfully.
